In [1]:
import json
import pandas as pd
from neo4j import GraphDatabase

URI = "neo4j://localhost"
AUTH = ("neo4j", "167trungkinh")

URI="neo4j+s://18429959.databases.neo4j.io"
AUTH = ("neo4j","6hXdC44Aa0Tm9UD3s5xGve3itUUNH9afYOcFOp2Z3ic")

### 1. Load Data from CSV file

In [2]:
# Load file entities
file_path = 'data/processed/file_entities.csv'
data = pd.read_csv(file_path)
for i in range(len(data['Name'])):
    data['Name'][i] = data['Name'][i].replace("'", " ")
    data['ReportName'][i] = str(data['ReportName'][i]).replace("'", " ")
# Load Basin entities
file_path = 'data/processed/basin_entities.csv'
basin_data = pd.read_csv(file_path)
# Load Block entities
file_path = 'data/processed/block_entities.csv'
block_data = pd.read_csv(file_path)
# Load Well entities
file_path = 'data/processed/well_entities.csv'
well_data = pd.read_csv(file_path)
# Load Author entities
file_path = 'data/processed/author_entities.csv'
author_data = pd.read_csv(file_path)
# Load Contractor entities
file_path = 'data/processed/contractor_entities.csv'
contractor_data = pd.read_csv(file_path)
# Load TypeofFormat entities
file_path = 'data/processed/type_of_format_entities.csv'
type_of_format_data = pd.read_csv(file_path)
# Load TypesOfReport entities
file_path = 'data/processed/types_of_report_entities.csv'
types_of_report_data = pd.read_csv(file_path)
# Load StatusOfReport entities
file_path = 'data/processed/status_of_report_entities.csv'
status_of_report_data = pd.read_csv(file_path)
# Load Fields entities
file_path = 'data/processed/fields_entities.csv'
fields_data = pd.read_csv(file_path)
# Load Function entities
file_path = 'data/processed/function_entities.csv'
function_data = pd.read_csv(file_path)
# Load TypeOfFile entities
file_path = 'data/processed/type_of_file_entities.csv'
type_of_file_data = pd.read_csv(file_path)

/var/folders/r3/x33m7bkj4dxcdt5jwwgg27xc0000gn/T/ipykernel_15788/907698370.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Name'][i] = data['Name'][i].replace("'", " ")
/var/folders/r3/x33m7bkj4dxcdt5jwwgg27xc0000gn/T/ipykernel_15788/907698370.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ReportName'][i] = str(data['ReportName'][i]).replace("'", " ")


### 2. Load Neo4j Nodes
Using Cypher Query `MERGE (p:Label name:{}, other_properties:{})` to create new Nodes. If node is existed, ignore the creation

#### 2.1 File Entities

In [3]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(data['Name'])):
        query = "MERGE (p:File {{\
                name: '{}',\
                id: '{}',\
                reportName: '{}',\
                title: '{}',\
                YearOfPublication: '{}',\
                LinkingUri: '{}',\
                LinkingUrl: '{}',\
                ServerRelativeUrl: '{}',\
                ServerRedirectedEmbedUri: '{}',\
                ServerRedirectedEmbedUrl: '{}',\
                Created: '{}',\
                Length: {}}})".format(
            data['Name'][i], 
            data['GUID'][i], 
            data['ReportName'][i], 
            data['Title'][i], 
            data['YearOfPublication'][i], 
            data['LinkingUri'][i], 
            data['LinkingUrl'][i], 
            data['ServerRelativeUrl'][i], 
            data['ServerRedirectedEmbedUri'][i], 
            data['ServerRedirectedEmbedUrl'][i], 
            data['Created'][i], 
            data['Length'][i],
            )
        records, summary, keys = driver.execute_query(query_=query, database_="neo4j")
        # Summary information
        print("The query `{query}` returned {records_count} records in {time} ms.".format(
            query=summary.query, records_count=len(records),
            time=summary.result_available_after
        ))
    


The query `MERGE (p:File {                name: '2008_BC Hop tac nghien cuu chung VN-TQ_ICB_Vol1_2.pdf',                id: '2eb71c4b-1968-440f-900a-fd7e65d055f3',                reportName: 'ICB-CCOP_SongHong - Yinggehai Basin Case Study',                title: 'nan',                YearOfPublication: '2008',                LinkingUri: 'nan',                LinkingUrl: 'nan',                ServerRelativeUrl: '/sites/Eptesting/Shared Documents/General/DATABASE/Song Hong Basin/1. Regional Studies/2008_BC Hop tac nghien cuu chung VN-TQ_ICB_Vol1_2.pdf',                ServerRedirectedEmbedUri: 'https://viendaukhivn.sharepoint.com/sites/Eptesting/_layouts/15/Embed.aspx?UniqueId=30590ef1-75c4-4d8f-a494-7e4950a5ae55',                ServerRedirectedEmbedUrl: 'https://viendaukhivn.sharepoint.com/sites/Eptesting/_layouts/15/Embed.aspx?UniqueId=30590ef1-75c4-4d8f-a494-7e4950a5ae55',                Created: '2024-01-18T10:41:59',                Length: 13090660})` returned 0 records in 36 ms.
T

#### 2.2 Create Other Nodes

In [4]:
def create_nodes(driver, query):
    records, summary, keys = driver.execute_query(query_=query, database_="neo4j")
    # Summary information
    print("The query `{query}` returned {records_count} records in {time} ms.".format(
        query=summary.query, records_count=len(records),
        time=summary.result_available_after
    ))

In [5]:
# Basin
basins = basin_data['Basin'].unique()
basins = basins[~pd.isnull(basins)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(basins)):
        query = "MERGE (p:Basin {{name: '{}'}})".format(basins[i])
        create_nodes(driver, query)

# Block
blocks = block_data['Block'].unique()
blocks = blocks[~pd.isnull(blocks)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(blocks[1:])):
        query = "MERGE (p:Block {{name: '{}'}})".format(blocks[i])
        create_nodes(driver, query)

# Well
wells = well_data['WellName'].unique()
wells = wells[~pd.isnull(wells)]
wells = wells.tolist()
wells.remove('Well Completion Report')
wells.remove('103-TH-1X and 103-TG-1X')
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(wells)):
        query = "MERGE (p:Well {{name: '{}'}})".format(wells[i])
        create_nodes(driver, query)


# Author
authors = author_data['ReportAuthor'].unique()
authors = authors[~pd.isnull(authors)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(authors)):
        query = "MERGE (p:Author {{name: '{}'}})".format(authors[i])
        create_nodes(driver, query)


# Contractor
contractors = contractor_data['Contractors'].unique()
contractors = contractors[~pd.isnull(contractors)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(contractors)):
        query = "MERGE (p:Contractor {{name: '{}'}})".format(contractors[i])
        create_nodes(driver, query)


# TypeOfFormat
type_of_formats = type_of_format_data['TypeofFormat'].unique()
type_of_formats = type_of_formats[~pd.isnull(type_of_formats)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(type_of_formats)):
        query = "MERGE (p:TypeOfFormat {{name: '{}'}})".format(type_of_formats[i])
        create_nodes(driver, query)


# TypesOfReport
types_of_reports = types_of_report_data['TypesOfReport'].unique()
types_of_reports = types_of_reports[~pd.isnull(types_of_reports)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(types_of_reports)):
        query = "MERGE (p:TypesOfReport {{name: '{}'}})".format(types_of_reports[i])
        create_nodes(driver, query)


# StatusOfReport
status_of_reports = status_of_report_data['StatusOfReport'].unique()
status_of_reports = status_of_reports[~pd.isnull(status_of_reports)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(status_of_reports)):
        query = "MERGE (p:StatusOfReport {{name: '{}'}})".format(status_of_reports[i])
        create_nodes(driver, query)

        
# Fields
fields = fields_data['Fields'].unique()
fields = fields[~pd.isnull(fields)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(fields)):
        query = "MERGE (p:Fields {{name: '{}'}})".format(fields[i])
        create_nodes(driver, query)


# Function
functions = function_data['Function'].unique()
functions = functions[~pd.isnull(functions)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(functions)):
        query = "MERGE (p:Function {{name: '{}'}})".format(functions[i])
        create_nodes(driver, query)


# TypeOfFile
type_of_files = type_of_file_data['TypeOfFile'].unique()
type_of_files = type_of_files[~pd.isnull(type_of_files)]
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for i in range(len(type_of_files)):
        query = "MERGE (p:TypeOfFile {{name: '{}'}})".format(type_of_files[i])
        create_nodes(driver, query)


The query `MERGE (p:Basin {name: 'SongHong'})` returned 0 records in 17 ms.
The query `MERGE (p:Block {name: 'All'})` returned 0 records in 11 ms.
The query `MERGE (p:Block {name: '104-109'})` returned 0 records in 3 ms.
The query `MERGE (p:Block {name: '111'})` returned 0 records in 2 ms.
The query `MERGE (p:Block {name: '112'})` returned 0 records in 2 ms.
The query `MERGE (p:Block {name: '113'})` returned 0 records in 3 ms.
The query `MERGE (p:Block {name: '114'})` returned 0 records in 3 ms.
The query `MERGE (p:Block {name: '115'})` returned 0 records in 3 ms.
The query `MERGE (p:Block {name: '100-101'})` returned 0 records in 4 ms.
The query `MERGE (p:Block {name: '102-106'})` returned 0 records in 3 ms.
The query `MERGE (p:Block {name: '103-107'})` returned 0 records in 3 ms.
The query `MERGE (p:Block {name: '104'})` returned 0 records in 2 ms.
The query `MERGE (p:Block {name: '105'})` returned 0 records in 3 ms.
The query `MERGE (p:Block {name: '117'})` returned 0 records in 3 m

#### 2.3 Delete all Nodes

In [6]:
# with GraphDatabase.driver(URI, auth=AUTH) as driver:
#     for i in range(len(data['Name'])):
#         query = "MATCH (n) DETACH DELETE n"
#         records, summary, keys = driver.execute_query(query_=query, database_="neo4j")
#         # Summary information
#         print("The query `{query}` returned {records_count} records in {time} ms.".format(
#             query=summary.query, records_count=len(records),
#             time=summary.result_available_after
#         ))